# **Modelos de Regresión**

En este laboratorio se reforzará el uso de la biblioteca **scikit-learn** para implementar la regresión lineal simple y la regresión no lineal. Primero se trabajará un conjunto de datos relacionado con el consumo de combustible y la emisión de dióxido de carbono de los automóviles. Luego, se dividirán los datos en conjuntos de entrenamiento y prueba, se creará un modelo usando el conjunto de entrenamiento, se evaluará su modelo usando el conjunto de prueba y finalmente se usará el modelo para predecir el valor desconocido.

Si los datos muestran una tendencia con curvas, la regresión lineal no producirá resultados muy precisos en comparación con una regresión no lineal porque, como su nombre lo indica, la regresión lineal supone que los datos son lineales. Por este motivo, se reforzarán los conceptos sobre regresiones no lineales y se aplicará en un ejemplo con un conjunto de datos relacionado con el PIB de China desde 1960 hasta 2014.

## Tabla de Contenidos

1.  [Importar y Preparar los Datos de Análisis](#2)
2.  [Modelo de Regresión Simple](#4)
3.  [Modelos de Regresión No Lineales](#6)
4.  [Ejemplo de una Regresión No Lineal](#8)

# 1. Importar y Preparar los Datos de Análisis <a id="2"></a>

## Carga y Preparación de Datos

Instalar e importar Bibliotecas:

In [13]:
# Instalar bibliotecas específicas para trabajar en el laboratorio

#!pip install numpy
#!pip install pandas
#!pip install matplotlib
#!pip install scipy
#!pip install sklearn

Importar los paquetes de procesamiento y visualización de datos **pandas**, **numpy**, **matplotlib**, **sklearn** y **scipy**. No olvidar de poner `% matplotlib inline` para que las gráficas puedan aparecer en *Jupyter Notebook*.

In [14]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

## Leer el Conjunto de Datos

Actualmente, se tiene clasificado el apoyo que brindan los padres a estudiantes de secundaria en cinco grupos. Si la utilización de los datos demográficos puede servirnos para predecir a que grupo pertenecen, el establecimiento educativo podría tomar medidas oportunas para potenciar el apoyo en la relación entre estudiantes y padres, en beneficio del desempeño estudiantil de los hijos. Así, esto sería un problema de clasificación. Es decir, dado el conjunto de datos con etiquetas predefinidas, se necesita construir un modelo para predecir la clase de un caso nuevo o desconocido.

Para este caso, se usarán datos demográficos, como Edad, Género, Etnicidad y Nivel Educativo de los Padres.

El campo de destino, llamado `ParentalSupport`, tiene cinco valores posibles que corresponden a los cinco niveles de apoyo de los padres, de la siguiente manera:

0. Ninguno
1. Bajo
2. Moderado
3. Alto
4. Muy alto



Cargar los datos y guardarlos en el dataframe `df`:

In [15]:
# ruta de datos y leer los datos para guardarlos en df

path='datos/Student_performance_data _.csv'
df = pd.read_csv(path)

In [16]:
# mostrar los 5 primeros registros de conjunto de datos

df.head()

,StudentID,Age,Gender,Ethnicity,ParentalEducation,StudyTimeWeekly,Absences,Tutoring,ParentalSupport,Extracurricular,Sports,Music,Volunteering,GPA,GradeClass
0,1001,17,1,0,2,19.833723,7,1,2,0,0,1,0,2.929196,2.0
1,1002,18,0,0,1,15.408756,0,0,1,0,0,0,0,3.042915,1.0
2,1003,15,0,2,3,4.210570,26,0,2,0,0,0,0,0.112602,4.0
3,1004,17,1,0,3,10.028829,14,0,3,1,0,0,0,2.054218,3.0
4,1005,17,1,0,2,4.672495,17,1,3,0,0,0,0,1.288061,4.0


## Pre-procesamiento de los Datos

Primero se va a realizar un rápido análisis y visualización de los Datos. Se utilizara el metodo `df.value_counts()`, para observar cuántos clientes hay en cada clase en el conjunto de datos.

In [17]:
df['ParentalSupport'].value_counts()

ParentalSupport
2    740
3    697
1    489
4    254
0    212
Name: count, dtype: int64

Como se puede observar, hay 212 clientes en Ninguno (0), 254 clientes en Muy alto (4), 489 clientes en Bajo (1), 697 clientes en Alto (3) y 740 clientes en Moderado (2).